### Verificar Similitud

In [ ]:
import cv2

def calcular_porcentaje_similitud(imagen1, imagen2):
    # Cargar las imágenes
    img1 = cv2.imread(imagen1)
    img2 = cv2.imread(imagen2)

    # Verificar si las imágenes se cargaron correctamente
    if img1 is None or img2 is None:
        raise ValueError("No se pudieron cargar las imágenes.")

    # Verificar si las imágenes tienen el mismo tamaño
    if img1.shape != img2.shape:
        raise ValueError("Las imágenes no tienen el mismo tamaño.")

    # Calcular la diferencia absoluta entre las imágenes
    diferencia = cv2.absdiff(img1, img2)

    # Convertir la diferencia en escala de grises
    diferencia_gris = cv2.cvtColor(diferencia, cv2.COLOR_BGR2GRAY)

    # Calcular el porcentaje de similitud
    porcentaje_similitud = (1 - (diferencia_gris.sum() / 255) / (diferencia_gris.size)) * 100

    return porcentaje_similitud

def probar_similitud(imagen_nombre):
    imagen=imagen_nombre.split('.')[0]
    print("Similitud entre el original y el encriptado (%): ",calcular_porcentaje_similitud(imagen+'.jpg',imagen+'_encrip.jpg'))
    print("Similitud entre el original y el desencriptado (%): ",calcular_porcentaje_similitud(imagen+'.jpg',imagen+'_desencrip.jpg'))

In [ ]:
# EJEMPLO:
probar_similitud('ima002.jpg')

Similitud entre el original y el encriptado (%):  75.73728
Similitud entre el original y el desencriptado (%):  98.37205960784313


##Conectar Drive

###Encriptar y Desencriptar carpetas

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
from google.colab import drive

# Montar Google Drive en Colab
drive.mount('/content/drive')

def encriptar(numero, clave_publica):
    e, n = clave_publica
    numero_encriptado = pow(numero, e, n)
    return numero_encriptado

def desencriptar(numero_encriptado, clave_privada):
    d, n = clave_privada
    numero_desencriptado = pow(numero_encriptado, d, n)
    return numero_desencriptado

def encriptar_imagen(nombre_imagen,clave,carpeta_origen, carpeta_destino):
    ruta_imagen = os.path.join('/content/drive/MyDrive', carpeta_origen, nombre_imagen + ".jpg")
    img = cv2.imread(ruta_imagen)
    if img is None:
        print(f"No se pudo cargar la imagen: {nombre_imagen}.jpg")
        return
    # Inicializar la matriz encriptada
    matriz_encriptada = np.zeros_like(img, dtype=np.uint16)
    matriz_RSA= np.zeros_like(img, dtype=np.uint16)
    # Encriptar los canales de color (RGB) de cada píxel
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                pixel = img[i, j, k]
                pixel_encriptado = encriptar(int(pixel), clave)
                matriz_encriptada[i, j, k] = pixel_encriptado
                matriz_RSA[i, j, k] = pixel_encriptado
    # Guardar la matriz encriptada como una imagen en formato jpg
    imagen_encriptada = Image.fromarray(matriz_encriptada.astype(np.uint8))
    ruta_destino = os.path.join(carpeta_destino, nombre_imagen + '_encrip.jpg')
    imagen_encriptada.save(ruta_destino)
    return matriz_RSA

def desencriptar_imagen(nombre_imagen, clave, carpeta_origen, matriz,carpeta_destino):
    ruta_imagen = os.path.join('/content/drive/MyDrive', carpeta_origen, nombre_imagen + "_encrip.jpg")
    img = cv2.imread(ruta_imagen)
    orden=[2,1,0]
    # Inicializar la matriz desencriptada
    matriz_desencriptada = np.zeros_like(img, dtype=np.uint16)
    # Desencriptar los canales de color (RGB) de cada píxel
    for i in range(matriz.shape[0]):
        for j in range(matriz.shape[1]):
            for k in range(matriz.shape[2]):
                pixel = matriz[i, j, k]
                pixel_desencriptado = desencriptar(int(pixel), clave)
                matriz_desencriptada[i, j, orden[k]] = pixel_desencriptado
    # Guardar la matriz desencriptada como una imagen en formato jpg
    imagen_desencriptada = Image.fromarray(matriz_desencriptada.astype(np.uint8))
    ruta_destino = os.path.join(carpeta_destino, nombre_imagen + '_desencrip.jpg')
    imagen_desencriptada.save(ruta_destino)
    return

def pruebas_encriptación(carpeta_origen, carpeta_destino_encriptadas, carpeta_destino_desencriptadas):
    # Rutas completas de las carpetas de origen y destino en Google Drive
    carpeta_origen_completa = os.path.join('/content/drive/MyDrive', carpeta_origen)
    carpeta_destino_encriptadas_completa = os.path.join('/content/drive/MyDrive', carpeta_destino_encriptadas)
    carpeta_destino_desencriptadas_completa = os.path.join('/content/drive/MyDrive', carpeta_destino_desencriptadas)

    # Obtener la lista de todas las imágenes en la carpeta de origen en Google Drive
    lista_imagenes = [nombre_archivo for nombre_archivo in os.listdir(carpeta_origen_completa) if nombre_archivo.lower().endswith(".jpg")]

    # Encriptar y desencriptar todas las imágenes
    for imagen in lista_imagenes:
        imagen_path = os.path.join(carpeta_origen_completa, imagen)
        img = cv2.imread(imagen_path)
        matriz_RSA = np.zeros_like(img, dtype=np.uint16)
        matriz_RSA = encriptar_imagen(imagen.replace('.jpg', ""), clave_publica, carpeta_origen,carpeta_destino_encriptadas_completa)
        desencriptar_imagen(imagen.replace('.jpg', ""), clave_privada,carpeta_destino_encriptadas_completa, matriz_RSA, carpeta_destino_desencriptadas_completa)

    return

# Rutas de las carpetas de origen y destino en Google Drive
carpeta_origen = "Imagenes"
carpeta_destino_encriptadas = "ImagenesEncriptadas"
carpeta_destino_desencriptadas = "ImagenesDesencriptadas"

# Llamar a la función de pruebas_encriptación para automatizar el proceso
pruebas_encriptación(carpeta_origen, carpeta_destino_encriptadas, carpeta_destino_desencriptadas)

Mounted at /content/drive


###Resultados de similitud

In [ ]:
import cv2
import os

# Montar Google Drive en Colab
from google.colab import drive
drive.mount('/content/drive')

def calcular_porcentaje_similitud(imagen1, imagen2):
    # Cargar las imágenes
    img1 = cv2.imread(imagen1)
    img2 = cv2.imread(imagen2)

    # Verificar si las imágenes se cargaron correctamente
    if img1 is None or img2 is None:
        raise ValueError("No se pudieron cargar las imágenes.")

    # Verificar si las imágenes tienen el mismo tamaño
    if img1.shape != img2.shape:
        raise ValueError("Las imágenes no tienen el mismo tamaño.")

    # Calcular la diferencia absoluta entre las imágenes
    diferencia = cv2.absdiff(img1, img2)

    # Convertir la diferencia en escala de grises
    diferencia_gris = cv2.cvtColor(diferencia, cv2.COLOR_BGR2GRAY)

    # Calcular el porcentaje de similitud
    porcentaje_similitud = (1 - (diferencia_gris.sum() / 255) / (diferencia_gris.size)) * 100

    return porcentaje_similitud

def probar_similitud(carpeta_origen, carpeta_encriptadas, carpeta_desencriptadas):
    # Rutas completas de las carpetas en Google Drive
    carpeta_origen_completa = os.path.join('/content/drive/MyDrive', carpeta_origen)
    carpeta_encriptadas_completa = os.path.join('/content/drive/MyDrive', carpeta_encriptadas)
    carpeta_desencriptadas_completa = os.path.join('/content/drive/MyDrive', carpeta_desencriptadas)

    # Obtener una lista de todas las imágenes en la carpeta de origen en Google Drive
    lista_imagenes = [nombre_archivo for nombre_archivo in os.listdir(carpeta_origen_completa) if nombre_archivo.lower().endswith(".jpg")]
    porcentaje_similitud_encriptada=[]
    porcentaje_similitud_desencriptada=[]
    i=0
    # Calcular el porcentaje de similitud para cada imagen
    for imagen in lista_imagenes:
        imagen_path = os.path.join(carpeta_origen_completa, imagen)
        imagen_encriptada_path = os.path.join(carpeta_encriptadas_completa, imagen.replace('.jpg', '_encrip.jpg'))
        imagen_desencriptada_path = os.path.join(carpeta_desencriptadas_completa, imagen.replace('.jpg', '_desencrip.jpg'))
        i+=1
        try:
            similitud_encriptada = calcular_porcentaje_similitud(imagen_path, imagen_encriptada_path)
            similitud_desencriptada = calcular_porcentaje_similitud(imagen_path, imagen_desencriptada_path)
            porcentaje_similitud_encriptada.append(similitud_encriptada)
            porcentaje_similitud_desencriptada.append(similitud_desencriptada)
            print(f"Imagen {i}: {imagen}")
            print(f"Similitud con la imagen encriptada: {similitud_encriptada:.2f}%")
            print(f"Similitud con la imagen desencriptada: {similitud_desencriptada:.2f}%")
            print("-" * 30)
        except ValueError as e:
            print(f"Error en la imagen {imagen}: {e}")
            print("-" * 30)

    # Análisis de los porcentajes
    # Porcentaje promedio de la similitud entre las imagenes originales y sus encriptadas:
    print("****************** RESULTADOS FINALES ********************")
    promedio_total_encrip=sum(porcentaje_similitud_encriptada)/len(porcentaje_similitud_encriptada)
    promedio_total_desencrip=sum(porcentaje_similitud_desencriptada)/len(porcentaje_similitud_desencriptada)
    print("Cantidad de Muestra: ", len(porcentaje_similitud_encriptada))
    print("Porcentaje de similitud entre las imágenes originales y encriptadas: ",promedio_total_encrip,"%")
    print("Porcentaje de similitud entre las imágenes originales y desencriptadas: ",promedio_total_desencrip,"%")

# Rutas de las carpetas de origen, encriptadas y desencriptadas en Google Drive
carpeta_origen = "Imagenes"
carpeta_encriptadas = "ImagenesEncriptadas"
carpeta_desencriptadas = "ImagenesDesencriptadas"

# Llamar a la función para probar la similitud
probar_similitud(carpeta_origen, carpeta_encriptadas, carpeta_desencriptadas)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Imagen 1: texto1.jpg
Similitud con la imagen encriptada: 43.41%
Similitud con la imagen desencriptada: 98.85%
------------------------------
Imagen 2: im24.jpg
Similitud con la imagen encriptada: 73.44%
Similitud con la imagen desencriptada: 98.68%
------------------------------
Imagen 3: im17.jpg
Similitud con la imagen encriptada: 67.11%
Similitud con la imagen desencriptada: 99.14%
------------------------------
Imagen 4: 946.jpg
Similitud con la imagen encriptada: 45.83%
Similitud con la imagen desencriptada: 98.99%
------------------------------
Imagen 5: im25.jpg
Similitud con la imagen encriptada: 47.60%
Similitud con la imagen desencriptada: 99.35%
------------------------------
Imagen 6: im1.jpg
Similitud con la imagen encriptada: 41.29%
Similitud con la imagen desencriptada: 99.85%
------------------------------
Imagen 7: im18.jpg
Similitud con la i